In [2]:
import sys
sys.path.insert(0, '/Users/bytedance/LearningProjects/practice_shouqianba')
sys.path.insert(0, '/Users/bytedance/LearningProjects/DeepCTR')

from test_deepctr import *

DeepCTR version 0.8.7 detected. Your version is 0.8.5.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.8.7


In [3]:
data = pd.read_csv("/Users/bytedance/LearningProjects/dataset/movielens_sample.txt")

train, test = train_test_split(data, test_size=0.2, random_state=2020)

In [4]:
pd.unique(train['user_id'])

array([4627, 2496, 3630,  366, 1285, 5964, 4566, 4094, 1010, 2181,  506,
       2943, 1680, 4964, 1408, 4214, 4732, 2820,  635, 3590, 1601, 1283,
        716,  854, 3808, 5848, 3471, 2857, 4655, 4694, 4227, 5293,  829,
       3312, 5512, 2837, 2533,  678, 2753, 4284, 1706,  872, 3675,  615,
       4815, 1667, 1834, 2023, 1589, 2744, 3710, 1106, 5156, 2346, 5153,
       3618, 1866, 1472, 1120, 3483, 3558,  182, 3947, 1836, 1861, 3053,
       1884,  621,  785, 1685,  249, 2077, 5256, 4520, 2073, 3299, 2271,
       4947, 4725,  613, 2996, 1303, 2211, 5893, 6036, 3045, 5732, 4658,
       2174, 1755, 3274, 2590, 3123,   80, 1321,  753, 5389, 1841, 3589,
       1717, 3762, 2913, 4205, 1141, 4290, 1579, 3679, 2230, 1265, 1941,
       1391,  415, 4802, 6040, 4384, 5365, 1776,  471, 1591, 5831, 4973,
       2841, 4645, 3184, 3087, 3402,  425, 1869, 1724,  974, 3401, 1383,
       3436, 2050, 5720, 1015,  192, 4238, 5355, 2695, 3410,  284, 1164,
       3759, 1868, 5521,  910, 5626, 1908, 1078, 50

In [5]:
pd.unique(test['user_id'])

array([ 877, 4238, 1579, 2002,  881, 3242, 1752, 3910, 1391, 3705, 3315,
       3430, 4502, 5080, 5630, 1503, 3568, 1273, 2438, 5157,  546, 2067,
       5788, 5511, 2600, 5108, 5039, 5227, 2996, 4738, 4715,  517, 4889,
       5767, 4373, 5371,  380, 5746, 5365, 3868])

In [11]:
tf.add_n([1,2,3])

<tf.Tensor: id=13, shape=(), dtype=int32, numpy=6>

In [5]:

dnn_feature_columns = [
    SparseFeat("movie_id", len(pd.unique(data["movie_id"])), embedding_dim=16, use_hash=True),
    SparseFeat("user_id", len(pd.unique(data["user_id"])), embedding_dim=16, use_hash=True),
]
linear_feature_columns = dnn_feature_columns

In [6]:
feature_names = get_feature_names(dnn_feature_columns + linear_feature_columns)

In [7]:
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}
train_model_input['movie_id']

array([2478, 1278, 3256, 1077, 1794,  454, 3683,  529, 2953, 2020, 3354,
       1197, 1988, 2348,  198, 3186, 3016,  457, 1270, 2000, 1396,  480,
        737,  345,   61,  909, 3614,  436, 2146,  159, 3635, 1374,  904,
        520, 1801, 2396, 2266, 1972, 3045,  517, 1916, 3092,  532,  296,
       3181, 1240, 2288, 1012, 2594,  588, 3033, 3624, 3688, 1416, 1290,
       3374, 3948, 2278, 1097, 2561, 1580, 1089,  593, 2736, 2642, 1296,
       1831,  833, 2115, 2664, 2369,  788,  208, 2160, 1784,  235, 2671,
        380, 1100,   32, 3763, 2344,  916, 2144, 2085, 2762, 2115, 1009,
       1046, 1653, 3698, 2097, 2407, 2059, 2240,  434, 2657, 2289,   70,
       1352, 1337, 1387, 3175,  813, 1193, 3101, 2557, 2873, 2396, 1263,
       3160, 1302, 1208, 3224, 2918, 1892, 3882, 3481,  350, 2458, 1246,
        680, 1379, 2344, 1397,  837, 1252,  471, 1244,  597, 3897, 3256,
        468, 2724, 2546,  349, 1184,  761, 3543,  590, 1278, 2565, 2716,
       1894, 2151, 3508, 3354,  412,  474, 1198,   

In [70]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )

In [72]:
# 直接吃了177M内存
asizeof.asizeof(model)

229810016

dict

In [73]:
model.fit(train_model_input, train["rating"].values, batch_size=256, epochs=10, verbose=2, validation_split=0.2, )

Train on 128 samples, validate on 32 samples
Epoch 1/10


/Users/bytedance/miniconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/Users/bytedance/miniconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


128/128 - 1s - loss: 13.4138 - mse: 13.4138 - val_loss: 15.7048 - val_mse: 15.7048
Epoch 2/10
128/128 - 0s - loss: 13.3058 - mse: 13.3058 - val_loss: 15.5908 - val_mse: 15.5908
Epoch 3/10
128/128 - 0s - loss: 13.1876 - mse: 13.1876 - val_loss: 15.4656 - val_mse: 15.4656
Epoch 4/10
128/128 - 0s - loss: 13.0572 - mse: 13.0572 - val_loss: 15.3296 - val_mse: 15.3296
Epoch 5/10
128/128 - 0s - loss: 12.9153 - mse: 12.9153 - val_loss: 15.1826 - val_mse: 15.1826
Epoch 6/10
128/128 - 0s - loss: 12.7613 - mse: 12.7613 - val_loss: 15.0241 - val_mse: 15.0241
Epoch 7/10
128/128 - 0s - loss: 12.5945 - mse: 12.5945 - val_loss: 14.8535 - val_mse: 14.8535
Epoch 8/10
128/128 - 0s - loss: 12.4140 - mse: 12.4140 - val_loss: 14.6699 - val_mse: 14.6699
Epoch 9/10
128/128 - 0s - loss: 12.2188 - mse: 12.2188 - val_loss: 14.4726 - val_mse: 14.4726
Epoch 10/10
128/128 - 0s - loss: 12.0078 - mse: 12.0078 - val_loss: 14.2607 - val_mse: 14.2607


In [78]:
test_model_input_v_test = {
    "movie_id": np.array([1,2]),
    "user_id": np.array([3,4])
}
model.predict(test_model_input_v_test, batch_size=256)

array([[0.24125732],
       [0.17255694]], dtype=float32)

{'movie_id': array([1485, 1884, 2420, 1945,    4, 3062, 2021, 3108, 1535,  540, 2918,
        2690, 1960, 3102,   21, 1971, 1230, 3095, 1125, 3462,  218, 1019,
         156, 2174, 3147,  367, 1792, 1196, 2799, 1242, 1221,  105,  590,
         198, 3167, 3194, 3481, 1242, 1042, 1626]),
 'user_id': array([ 877, 4238, 1579, 2002,  881, 3242, 1752, 3910, 1391, 3705, 3315,
        3430, 4502, 5080, 5630, 1503, 3568, 1273, 2438, 5157,  546, 2067,
        5788, 5511, 2600, 5108, 5039, 5227, 2996, 4738, 4715,  517, 4889,
        5767, 4373, 5371,  380, 5746, 5365, 3868])}

In [ ]:
sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
    target = ['rating']

    # 1.Label Encoding for sparse features,and do simple Transformation for dense features
    for feat in sparse_features:
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])
    # 2.count #unique features for each sparse field
    fixlen_feature_columns = [SparseFeat(feat, data[feat].max() + 1,embedding_dim=4)
                              for feat in sparse_features]
    linear_feature_columns = fixlen_feature_columns
    dnn_feature_columns = fixlen_feature_columns
    feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

    # 3.generate input data for model
    train, test = train_test_split(data, test_size=0.2, random_state=2020)
    train_model_input = {name:train[name].values for name in feature_names}
    test_model_input = {name:test[name].values for name in feature_names}

    # 4.Define Model,train,predict and evaluate
    model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
    model.compile("adam", "mse", metrics=['mse'], )

    history = model.fit(train_model_input, train[target].values,
                        batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    print("test MSE", round(mean_squared_error(
        test[target].values, pred_ans), 4))